# Reproduce Figure 6

# Setup BART, cfl2png and inkscape (on Colab)

This notebook requires BART (v0.9.00) to perform data processing and reconstructions, cfl2png to generate PNGs from the reconstructions and inkscape to join the individual PNGs to the final figure.

If this notebook is run on Google Colab, the setup is performed in the following cells. If not, it is assumed that the tools are installed manually.

In [1]:
# environment to check if on Colab

import os

if os.getenv("COLAB_RELEASE_TAG"):
  os.environ["COLAB"] = "1"
else:
  os.environ["COLAB"] = "0"

Install dependencies

%%bash

[ $COLAB -ne 1 ] && echo "Skipp cell (not on Colab)" && exit 0

# Install BARTs dependencies
apt-get install -y make gcc libfftw3-dev liblapacke-dev libpng-dev libopenblas-dev &> /dev/null

# Install dependencies for view
apt-get install -y libgtk-3-dev &> /dev/null

# Install inkscape
apt-get install -y inkscape &> /dev/null

Clone BART from github, checkout v0.9.00 and build it.

In [2]:
%%bash

[ $COLAB -ne 1 ] && echo "Skipp cell (not on Colab)" && exit 0

# Clone Bart
[ -d bart ] && rm -r bart

git clone https://github.com/mrirecon/bart/ bart &> /dev/null

cd bart

git checkout v0.9.00

make &> /dev/null

Skipp cell (not on Colab)


Setup environment variables to find BART executable.

In [3]:
import os

if os.environ["COLAB"] == "1":
  os.environ['BART_TOOLBOX_PATH']=os.getcwd()+"/bart/"
  os.environ['PATH'] = os.environ['BART_TOOLBOX_PATH'] + ":" + os.environ['PATH']
  os.environ['PATH'] = os.getcwd()+"/view/" + ":" + os.environ['PATH']

Clone view (cfl2png) from github, checkout v0.3.00 and build it.

In [4]:
%%bash

[ $COLAB -ne 1 ] && echo "Skipp cell (not on Colab)" && exit 0

# Clone Bart
[ -d view ] && rm -r view

git clone https://github.com/mrirecon/view.git view &> /dev/null

cd view

git checkout v0.3.00

make &> /dev/null

Skipp cell (not on Colab)


Check Setup by generating Shepp-Logan Phantom and converting to PNG:

In [5]:
%%bash

echo "BART version: $(bart version)"

bart phantom phan
cfl2png -x1 -y0 phan phan

BART version: v0.9.00


![Phantom](phan.png)

# Download Raw Data from Zenodo

In [6]:
%%bash

#bart twixread -A meas_MID00709_FID13299_t2_tse_tra_R=1.dat ksp_all
#bart slice 13 11 ksp_all ksp_fully

if [ ! -f ksp_fully.cfl ] ; then

	LINK_TO_ZENODO=???
	wget $LINK_TO_ZENODO/ksp_fully.hdr
	wget $LINK_TO_ZENODO/ksp_fully.cfl
fi

# Fully Sampled and Subsampled Reconstructions: Part A

## Generate Subsampling Pattern and Undersample k-Space

In [7]:
%%bash

bart poisson -C24 -Y420 -y4 -Z24 tmp
bart slice 2 0 tmp pat

bart repmat 0 420 pat tmp
cfl2png -x 1 -y 0 tmp pat

bart fmac ksp_fully pat ksp_raw

points: 2965, grid size: 420x24 = 10080 (R = 3.399663)


## Prewhiten k-Space Data

Noise is extracted from a region outside the FOV. The same noise from the undersampled k-space data is used to whiten the the fully sampled and the undersampled k-space data.

In [8]:
%%bash

bart fft -i -u 1 ksp_raw tmp
bart slice 0 50 tmp tmp
bart transpose 1 0 tmp noise

bart whiten -n ksp_raw noise ksp_white

bart whiten -n ksp_fully noise ksp_fully_white

## Fully Sampled Reference Reconstruction

In [23]:
%%bash

bart ecalib -m1 ksp_fully_white col

bart fft -i -u 3 ksp_fully_white cim_os
bart fmac -C -s8 cim_os col img
bart flip 1 img img

bart resize -c 0 320 1 320 img img
bart rss 8 img rss

cfl2png -A -u800 -x1 -y0 rss rss

Done.


<img src="rss.png" alt="Fully Sampled Reference" width="300"/>

## Coil Compression to 12 Virtual Coils

In [10]:
%%bash

bart cc -p 12 ksp_white ksp_cc

Done.


## Remove Frequency Oversampling

In [11]:
%%bash

# Remove frequency oversampling
bart fft -u -i 1 ksp_cc tmp
bart resize -c 0 320 tmp tmp
bart fft -u 1 tmp ksp_nos

## Reference Reconstruction of Undersampled k-Space Data

In [24]:
%%bash

POST=_rec

# Estimate Coil Sensitivities
bart ecalib -m1 ksp_cc tmp
bart resize -c 0 320 tmp col

# Perform PICS reconstruction
bart pics -S -RW:3:0:0.001 -i300 ksp_nos col img
bart flip 1 img img

# Take magnitude
bart rss 0 img rss$POST

# Remove phase oversampling
bart resize -c 0 320 1 320 rss$POST rss$POST

# Compute difference
bart saxpy -- -1 rss rss$POST diff$POST
bart rss 0 diff$POST diff$POST

cfl2png -A -u800 -x1 -y0 rss$POST rss$POST
cfl2png -A -u40  -x1 -y0 diff$POST diff$POST

Done.
Size: 134400 Samples: 37119 Acc: 3.62
l1-wavelet regularization: 0.001000
Regularization terms: 1, Supporting variables: 0
FISTA
Total Time: 20.423241



<img src="rss_rec.png" alt="Reconstruction" width="300"/>
<img src="diff_rec.png" alt="Difference to Fully Sampled" width="300"/>

# Reproduce the Reconstructions: Part B

## Missing Information on Number of Iterations

In [25]:
%%bash

POST=_iter

# Estimate Coil Sensitivities
bart ecalib -m1 ksp_cc tmp
bart resize -c 0 320 tmp col

# Perform PICS reconstruction
bart pics -S -RW:3:0:0.001 -i100 ksp_nos col img
bart flip 1 img img

# Take magnitude
bart rss 0 img rss$POST

# Remove phase oversampling
bart resize -c 0 320 1 320 rss$POST rss$POST

# Compute difference
bart saxpy -- -1 rss_rec rss$POST diff$POST
bart rss 0 diff$POST diff$POST

cfl2png -A -u800 -x1 -y0 rss$POST rss$POST
cfl2png -A -u40  -x1 -y0 diff$POST diff$POST

Done.
Size: 134400 Samples: 37119 Acc: 3.62
l1-wavelet regularization: 0.001000
Regularization terms: 1, Supporting variables: 0
FISTA
Total Time: 8.134389



<img src="rss_iter.png" alt="Reconstruction" width="300"/>
<img src="diff_iter.png" alt="Difference to Reproduced Reconstruction" width="300"/>

## Missing Information on Optimization Algorithm (not shown)

In [26]:
%%bash

POST=_algo

# Estimate Coil Sensitivities
bart ecalib -m1 ksp_cc tmp
bart resize -c 0 320 tmp col

# Perform PICS reconstruction
bart pics -S -RW:3:0:0.001 -i300 -m ksp_nos col img
bart flip 1 img img

# Take magnitude
bart rss 0 img rss$POST

# Remove phase oversampling
bart resize -c 0 320 1 320 rss$POST rss$POST

# Compute difference
bart saxpy -- -1 rss_rec rss$POST diff$POST
bart rss 0 diff$POST diff$POST

cfl2png -A -u800 -x1 -y0 rss$POST rss$POST
cfl2png -A -u40  -x1 -y0 diff$POST diff$POST

Done.
Size: 134400 Samples: 37119 Acc: 3.62
l1-wavelet regularization: 0.001000
Regularization terms: 1, Supporting variables: 0
ADMM
Total Time: 36.124603



<img src="rss_algo.png" alt="Reconstruction" width="300"/>
<img src="diff_algo.png" alt="Difference to Reproduced Reconstruction" width="300"/>

## Missing Information on Number of Virtual Coils

In [27]:
%%bash

POST=_cc

bart cc -p 8 ksp_white ksp_cc2

# Remove frequency oversampling
bart fft -u -i 1 ksp_cc2 tmp
bart resize -c 0 320 tmp tmp
bart fft -u 1 tmp ksp_nos2

# Estimate Coil Sensitivities
bart ecalib -m1 ksp_cc2 tmp
bart resize -c 0 320 tmp col

# Perform PICS reconstruction
bart pics -S -RW:3:0:0.001 -i300 ksp_nos2 col img
bart flip 1 img img

# Take magnitude
bart rss 0 img rss$POST

# Remove phase oversampling
bart resize -c 0 320 1 320 rss$POST rss$POST

# Compute difference
bart saxpy -- -1 rss_rec rss$POST diff$POST
bart rss 0 diff$POST diff$POST

cfl2png -A -u800 -x1 -y0 rss$POST rss$POST
cfl2png -A -u40  -x1 -y0 diff$POST diff$POST

Done.
Done.
Size: 134400 Samples: 37119 Acc: 3.62
l1-wavelet regularization: 0.001000
Regularization terms: 1, Supporting variables: 0
FISTA
Total Time: 22.132471



<img src="rss_cc.png" alt="Reconstruction" width="300"/>
<img src="diff_cc.png" alt="Difference to Reproduced Reconstruction" width="300"/>

## Missing Information on ESPIRiT Parameter

In [28]:
%%bash

POST=_coils

# Estimate Coil Sensitivities
# WARNING: on Debian 12 the standard OpenBLAS library triggers a segfault in the SVD
#          (c.f. https://github.com/OpenMathLib/OpenBLAS/issues/5000)
#          As a workaround, the BLAS backend of BART can be changed with
#          $ sudo update-alternatives --config libblas.so.3-x86_64-linux-gnu
#          and selecting the slower reference BLAS implementation.

bart ecalib -a -m1 ksp_cc tmp
bart resize -c 0 320 tmp col

# Perform PICS reconstruction
bart pics -S -RW:3:0:0.001 -i300 ksp_nos col img
bart flip 1 img img

# Take magnitude
bart rss 0 img rss$POST

# Remove phase oversampling
bart resize -c 0 320 1 320 rss$POST rss$POST

# Compute difference
bart saxpy -- -1 rss_rec rss$POST diff$POST
bart rss 0 diff$POST diff$POST

cfl2png -A -u800 -x1 -y0 rss$POST rss$POST
cfl2png -A -u40  -x1 -y0 diff$POST diff$POST


---------------------------------------------
| CTR1 | CTR2 |  Crop  |      Est. MSE      |
---------------------------------------------
|    1 |    1 | 0.9900 | 1.660466700000e+07 |
|      |    2 | 0.8900 | 1.456901300000e+07 |
|      |    3 | 0.7900 | 1.410630200000e+07 |
|      |    4 | 0.6900 | 1.440761100000e+07 |
|    2 |    1 | 0.7400 | 1.426520600000e+07 |
|      |    2 | 0.7900 | 1.410630200000e+07 |
|      |    3 | 0.8400 | 1.415684100000e+07 |
|    3 |    1 | 0.8150 | 1.407813500000e+07 |
|      |    2 | 0.7900 | 1.410630200000e+07 |
|    4 |    1 | 0.8025 | 1.408442900000e+07 |
|      |    2 | 0.8150 | 1.407813500000e+07 |
|      |    3 | 0.8275 | 1.408267400000e+07 |
|    5 |    1 | 0.8212 | 1.407918000000e+07 |
|      |    2 | 0.8150 | 1.407813500000e+07 |
|      |    3 | 0.8087 | 1.407986600000e+07 |
|    6 |    1 | 0.8119 | 1.407812500000e+07 |
|      |    2 | 0.8150 | 1.407813500000e+07 |
|    7 |    1 | 0.8134 | 1.407773700000e+07 |
|      |    2 | 0.8119 | 1.4078125


<img src="rss_coils.png" alt="Reconstruction" width="300"/>
<img src="diff_coils.png" alt="Difference to Reproduced Reconstruction" width="300"/>

## Missing Information on Normalization of k-Space Data

In [29]:
%%bash

POST=_normalize

# Estimate Coil Sensitivities
bart ecalib -m1 ksp_cc tmp
bart resize -c 0 320 tmp col

# Perform PICS reconstruction
bart pics -S -RW:3:0:0.001 -i300 -w1. ksp_nos col img
bart flip 1 img img

# Take magnitude
bart rss 0 img rss$POST

# Remove phase oversampling
bart resize -c 0 320 1 320 rss$POST rss$POST

# Compute difference
bart saxpy -- -1 rss_rec rss$POST diff$POST
bart rss 0 diff$POST diff$POST

cfl2png -A -u800 -x1 -y0 rss$POST rss$POST
cfl2png -A -u40  -x1 -y0 diff$POST diff$POST

Done.
Size: 134400 Samples: 37119 Acc: 3.62
l1-wavelet regularization: 0.001000
Regularization terms: 1, Supporting variables: 0
FISTA
Total Time: 25.006019



<img src="rss_normalize.png" alt="Reconstruction" width="300"/>
<img src="diff_normalize.png" alt="Difference to Reproduced Reconstruction" width="300"/>

## Missing Information on Prewhitening (not shown)

In [30]:
%%bash

POST=_white

bart cc -p 12 ksp_raw ksp_cc2

# Remove frequency oversampling
bart fft -u -i 1 ksp_cc2 tmp
bart resize -c 0 320 tmp tmp
bart fft -u 1 tmp ksp_nos2

# Estimate Coil Sensitivities
bart ecalib -m1 ksp_cc2 tmp
bart resize -c 0 320 tmp col

# Perform PICS reconstruction
bart pics -S -RW:3:0:0.001 -i300 ksp_nos2 col img
bart flip 1 img img

# Take magnitude
bart rss 0 img rss$POST

# Remove phase oversampling
bart resize -c 0 320 1 320 rss$POST rss$POST

bart nrmse -s rss$POST rss_rec
bart scale 960703.75 rss$POST rss$POST

# Compute difference
bart saxpy -- -1 rss_rec rss$POST diff$POST
bart rss 0 diff$POST diff$POST

cfl2png -A -u800 -x1 -y0 rss$POST rss$POST
cfl2png -A -u40  -x1 -y0 diff$POST diff$POST

Done.
Done.
Size: 134400 Samples: 37119 Acc: 3.62
l1-wavelet regularization: 0.001000
Regularization terms: 1, Supporting variables: 0
FISTA
Total Time: 21.740030
Scaled by: 960704.062500+0.000000i
0.161680



<img src="rss_white.png" alt="Reconstruction" width="300"/>
<img src="diff_white.png" alt="Difference to Reproduced Reconstruction" width="300"/>

# Generate Final Figure

In [ ]:
%%bash

if [ ! -f fig_6_template.svg ]; then
	LINK_TO_GITHUB=???
	wget $LINK_TO_GITHUB/figures/Fig_6_template.svg
fi

inkscape --export-filename=Fig_6_reproducible_recon.pdf Fig_6_template.svg